In [19]:
from dash import Dash, html, dash_table,dcc,Input,Output,State,callback
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
from scholarly import scholarly

In [20]:
%run mysql_utils.ipynb

In [21]:
sql_conn = get_mysql_db()

# Skip below parts when running because they are already run

In [7]:
cur = sql_conn.cursor()
sql_index = "create index year_idx on publication(year);"
cur.execute(sql_index)
sql_conn.commit()
cur.close()

ProgrammingError: 1061 (42000): Duplicate key name 'year_idx'

In [ ]:
sql_show_index = "show index from publication"
pd.read_sql(sql_show_index,sql_conn)

In [ ]:
cur = sql_conn.cursor()
sql_view = "create view pub_year_key as \
select count(distinct p.id) as pub_count,k.name as keyword,p.year \
from publication as p \
join Publication_Keyword as pk on pk.publication_id = p.id \
join keyword as k on k.id = pk.keyword_id \
group by k.name,p.year order by count(distinct p.id) desc;"
cur.execute(sql_view)
sql_conn.commit()
cur.close()

In [ ]:
sql_show_view = "select * from pub_year_key limit 1"
pd.read_sql(sql_show_view,sql_conn)

In [ ]:
cur = sql_conn.cursor()
sql_trigger = "create trigger ResetYear before update on publication \
for each row begin \
if new.year > 2023 then set new.year = 2023; end if; end"
sql_conn.commit()
cur.close()

In [300]:
cur = sql_conn.cursor()
sql_constraint = "alter table faculty_keyword add foreign key \
(keyword_id) references keyword(id);"
sql_conn.commit()
cur.close()

True

# Run from below

In [22]:
def get_sql_1(year1,year2):
    sql_1 = "select sum(pub_count) as 'Number of publications',keyword \
    from pub_year_key \
    where year >= {0} and year <= {1} group by keyword \
    order by sum(pub_count) desc limit 10".format(year1,year2)
    df_sql_1 = pd.read_sql(sql_1,sql_conn)
    return df_sql_1
#get_sql_1(2010,2022)

In [23]:
sql_2 = "select distinct(year),count(distinct p.id) as pub_count from publication as p group by year"
df_sql_2 = pd.read_sql(sql_2,sql_conn)
df_sql_2

/tmp/ipykernel_205515/1561136892.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,pub_count
0,0,119
1,1903,1
2,1907,1
3,1908,1
4,1909,1
...,...,...
95,2017,27430
96,2018,28896
97,2019,30621
98,2020,30575


In [24]:
sql_keyword = "select distinct name as keyword from keyword"
df_sql_keyword = pd.read_sql(sql_keyword,sql_conn)
df_sql_keyword= df_sql_keyword.sort_values('keyword')
df_sql_keyword

/tmp/ipykernel_205515/2929595335.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,keyword
25901,1-planar graphs
18570,2-factor
28347,20th century
22716,3-connected graph
28745,3-manifolds
...,...
6745,zipf distribution
19864,zone
23576,zone routing protocol
25846,zones


In [25]:
%run mongodb_utils.ipynb

In [26]:
mongo_conn = get_mongo_db()

In [27]:
"""
mongo_query = ([
{'$match':{'$and':[{"year":{'$gte':2000}},
{"year":{'$lte':2012}}]}},{'$unwind':"$keywords"},
{'$group':{'_id':"$keywords.name","pub_count":{'$sum':1}}},
{'$sort':{"pub_count":-1}},{'$limit':10}])
mongo_data = mongo_conn.publications.aggregate(mongo_query)
"""

'\nmongo_query = ([\n{\'$match\':{\'$and\':[{"year":{\'$gte\':2000}},\n{"year":{\'$lte\':2012}}]}},{\'$unwind\':"$keywords"},\n{\'$group\':{\'_id\':"$keywords.name","pub_count":{\'$sum\':1}}},\n{\'$sort\':{"pub_count":-1}},{\'$limit\':10}])\nmongo_data = mongo_conn.publications.aggregate(mongo_query)\n'

In [28]:
def get_mongo_1(keyword):
    mongo_query_2 = ([
    {'$unwind':"$keywords"},
    {'$match':{"keywords.name":keyword}},
    {'$group':{'_id':"$year","pub_count":{'$sum':1}}},
    {'$sort':{"pub_count":-1}}])
    mongo_data = mongo_conn.publications.aggregate(mongo_query_2)
    mongo_data_df = pd.DataFrame(list(mongo_data))
    return mongo_data_df

In [29]:
get_mongo_1("internet")

,_id,pub_count
0,2018,458
1,2017,433
2,2019,427
3,2004,424
4,2002,415
5,2010,408
6,2007,401
7,2005,398
8,2016,394
9,2003,389


In [30]:
%run neo4j_utils.ipynb

In [31]:
def get_neo_1(keyword):
    query = '''
    match (k:KEYWORD)<- [r2:LABEL_BY]-(p:PUBLICATION)<-[r:PUBLISH]
    -(f:FACULTY)-[r3:AFFILIATION_WITH]->(i:INSTITUTE) 
    where k.name in {0}
    return f.photoUrl as photo,f.name as name,i.name as university,
    count(p) as Num_pub,sum(p.numCitations*r2.score) as KRC order by 
    sum(p.numCitations*r2.score) DESC limit(5)
    '''.format([keyword])
    neo_data = get_neo4j_data(query)
    neo_data['KRC'] = neo_data['KRC'].astype(int)
    for index,value in enumerate(neo_data.photo):
        neo_data.photo[index] = html.Img(src=value,height=200,width=200)
    neo_data = neo_data.rename(columns={"Num_pub":"Number of Publications",
                               "KRC":"Keyword relevance score"})
    return neo_data

In [32]:
neo_data = get_neo_1('data mining')

/tmp/ipykernel_205515/412598113.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
neo_data

,photo,name,university,Number of Publications,Keyword relevance score
0,[],Jiawei Han,University of illinois at Urbana Champaign,116,14354
1,[],Vipin Kumar,University of Minnesota--Twin Cities,53,12763
2,[],Padhraic Smyth,University of California--Irvine,20,7200
3,[],George Karypis,University of Minnesota--Twin Cities,27,4031
4,[],Matei Zaharia,Stanford University,2,4019


In [34]:
def get_neo_2(keyword):
    query = '''
    match (k:KEYWORD)<- [r2:LABEL_BY]-
    (p:PUBLICATION)<-[r:PUBLISH]-(f:FACULTY)-[r3:AFFILIATION_WITH]->
    (i:INSTITUTE) 
    where k.name in {0}
    return i.name as University,count(distinct f.id) as Num_faculty
    order by count(distinct f.id) DESC limit(5)
    '''.format([keyword])
    neo_data_2 = get_neo4j_data(query)
    neo_data_2 = neo_data_2.rename(columns={"Num_faculty":"Number of faculty"})
    return neo_data_2

In [35]:
get_neo_2('data mining')

,University,Number of faculty
0,University of illinois at Urbana Champaign,39
1,University of Southern California,27
2,Carnegie Mellon University,23
3,Purdue University--West Lafayette,23
4,Northeastern University,23


In [36]:

app = Dash(__name__)
colors = {'background':'#EEE8CD','text':'#1A1A1A'}

app.layout = html.Div(\
    style={'backgroundColor':colors['background']},
    children=[
        html.H1(children='Popular research topics and topic related comparison',
                     style={'textAlign':'center','color':colors['text']}),
        html.Hr(),
        html.Div(children=['''
        choose year range from:
        ''',
        dcc.Dropdown(df_sql_2.year.unique(),'2020',id='from_year'),
                           '''to:''',
        dcc.Dropdown(df_sql_2.year.unique(),'2020',id='to_year'),
        dcc.Graph(id = 'bar_keywords')]),
        html.Hr(),
        
        #dash_table.DataTable(data=df_sql_1.to_dict('records'),
        #                    page_size =10),
        html.Div(children=['''
        choose keyword:
        ''',dcc.Dropdown(df_sql_keyword.keyword.unique(),
                         'machine learning',
                     id='keyword_name'),
        dcc.Graph(id = 'keyword_trend')]),
        html.Hr(),
        html.Div(id='datatable2'),
        html.Hr(),
        html.Div(dcc.Graph(id='bar_2')),
        html.Hr(),
        html.Div(['Add keyword:',dcc.Input(id='add_key',
                                           value='Type to add keyword',
                                           type='text'),
                 html.Button(id="add_button",type='submit',
                             children='Add'),
                 html.Div(id='output_add')]),
        html.Div(['Delete keyword:',dcc.Input(id='delete_key',
                                             value='Type to delete keyword',
                                             type='text'),
                 html.Button(id="delete_button",type='submit',
                             children='Delete'),
                 html.Div(id='output_delete')]),
        html.Div(['Show me current affiliation and photo for:',dcc.Input(id='faculty',
                                                  value='Type name of faculty',
                                                  type='text'),
                 html.Button(id="show_button",type='submit',
                            children="Search"),
                 html.Div(id='output_faculty'),
                 html.Div(['Would you like to update backend data? ',
                          html.Button(id='update_button',type='submit',
                                     children="Yes, update")])
                 ])
        
        
        
        
        
    ])
@callback(
    Output('bar_keywords','figure'),
    Input('from_year','value'),
    Input('to_year','value'))

def update_bar(from_year,to_year):
    from_year = min(from_year,to_year)
    to_year = max(from_year,to_year)
    df = get_sql_1(from_year,to_year)
    fig = px.bar(df,x=df.keyword,y=df['Number of publications'],
                title='Top 10 keywords from {0} to {1}'.format(from_year,to_year))

    return fig

@callback(
    Output('keyword_trend','figure'),
    
    Input('keyword_name','value')
)
def update_keyword(keyword_name):
    df1 = get_mongo_1(keyword_name).sort_values('_id')
    fig = px.line(df1,x=df1._id,y=df1.pub_count,title='Publication trend for {}'.format(keyword_name),
                )
    fig.update_layout( xaxis_title='Year',yaxis_title='Number of Publications')
    
    
    return fig

@callback(Output('datatable2','children'),
         Input('keyword_name','value'))
def update_datatable(keyword_name):
    df2 = get_neo_1(keyword_name)
    return dbc.Table.from_dataframe(df2)

@callback(
    Output('bar_2','figure'),
    Input('keyword_name','value'))
def update_bar_2(keyword_name):
    df = get_neo_2(keyword_name)
    fig = px.bar(df,x=df['Number of faculty'],y=df.University,
                title='Top 10 University with most staff working on {}'.format(keyword_name)
                )

    return fig

@callback(
    Output('output_add','children'),
    [Input('add_button','n_clicks')],
    [State('add_key','value')])
def add_key(clicks,input_value):
    if clicks is not None:
        while clicks>0:
            clicks-=1
            sql_keyword = "select distinct name as keyword from keyword"
            df_sql_keyword = pd.read_sql(sql_keyword,sql_conn)
            if input_value in df_sql_keyword.keyword.values:
                return "This keyword already exists! Please check spelling and re-enter."
            max_keyword_id_query = "select max(id) from keyword"
            df_id = pd.read_sql(max_keyword_id_query,sql_conn)
            max_id = df_id.iloc[0,0]
            cur = sql_conn.cursor()
            query = "insert into keyword (id,name) values ({0},'{1}')".format(max_id+1,input_value)   
            cur.execute(query)
            sql_conn.commit()
            cur.close()
        return "{} added".format(input_value)
            
        
@callback(
    Output('output_delete','children'),
    [Input('delete_button','n_clicks')],
    [State('delete_key','value')])
def delete_key(clicks,input_value):
    if clicks is not None:
        
        while clicks>0:
            clicks-=1
            sql_keyword = "select distinct name as keyword from keyword"
            df_sql_keyword = pd.read_sql(sql_keyword,sql_conn)
            if input_value not in df_sql_keyword.keyword.values:
                return "This keyword does not exist! Please check spelling and re-enter."

            cur = sql_conn.cursor()
            query = "delete from keyword where name = '{0}'".format(input_value)   
            cur.execute(query)
            sql_conn.commit()
            cur.close()
        return "{} deleted".format(input_value)
    
@callback(Output('output_faculty','children'),
          [Input('show_button','n_clicks')],
    [State('faculty','value')],
         [Input('update_button','n_clicks')])
def update_faculty(clicks,input_value,clicks_2):
    if clicks is not None:
        search_query = scholarly.search_author('{}'.format(input_value))
        first_author_result = next(search_query)
        photo = html.Img(src=first_author_result['url_picture'],height=200,width=200)
        affiliation = first_author_result['affiliation']
        interests =  first_author_result['interests']
        df = pd.DataFrame(columns=['photo','affiliation','interests'],data=[[photo,affiliation,interests]])
        if clicks_2 is not None:
            univ_query = "select id from university where name = '{}'".format(affiliation)
            univ_id = pd.read_sql(univ_query,sql_conn)
            if len(univ_id)!=0:
                uid = univ_id.iloc[0,0]
                update_query_1 ="update faculty set photo_url='{0}', university_id = {1}"\
                .format(first_author_result['url_picture'],uid)
            else:
                univ_max_id = "select max(id) from university"
                uid = pd.read_sql(univ_max_id,sql_conn).iloc[0,0]+1
                cur = sql_conn.cursor()
                query = "insert into university (id,name,photo_url) values ({0},'{1}',NULL)".format(uid,affiliation)   
                cur.execute(query)
                sql_conn.commit()
                cur.close()
                update_query_1 ="update faculty set photo_url='{0}', university_id = {1}"\
                .format(first_author_result['url_picture'],uid)
            cur = sql_conn.cursor()
            cur.execute(update_query_1)
            sql_conn.commit()
            cur.close()
            
        return dbc.Table.from_dataframe(df)


if __name__ == '__main__':
    app.run(debug=True,jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

/tmp/ipykernel_205515/3685725548.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/tmp/ipykernel_205515/412598113.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_205515/3685725548.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

